# Looking forward and looking back in Dutch newspapers

Code is designed for raw csv's from Delpher with 28 columns.

In [ ]:
import pandas as pd
import glob
import random
import os
import string
import re
import numpy as np
from collections import Counter
from datetime import datetime
from nltk import word_tokenize
from nltk.util import ngrams
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as mplcm
import matplotlib.colors as colors
from matplotlib.pyplot import cm
from tqdm.notebook import tqdm as log_progress

In [ ]:
paper = 'De Telegraaf'
paper_short = 'telegraaf'
path = "/path/to/folder/De_Telegraaf/"
allfiles = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
jaren = ['een', 'twee', 'drie', 'vier', 'vijf', 'zes', 'zeven', 'acht', 'negen', 'tien', 'elf', 'twaalf', 'dertien', 'veertien', 'vijftien', 'zestien', 'zeventien', 'achttien', 'negentien', 'twintig', 'dertig', 'veertig', 'vijftig', 'zestig', 'zeventig', 'tachtig', 'negentig', 'honderd', 'honderdtien', 'honderdtwintig', 'honderddertig', 'honderdveertig', 'honderdvijftig', 'honderdzestig', 'honderdzeventig', 'honderdtachtig', 'honderdnegentig', 'tweehonderd']
print(len(jaren))

In [ ]:
def query_column(i):
    """Function for naming df columns"""
    i_df = i.replace(' ','_')
    return i_df

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in word_tokenize(words):
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '' and len(new_word) >=3:
            new_words.append(new_word)
    words_str = (" ".join(new_words))
    return words_str

def word_count(string):
    """Function to count words in text string"""
    words = string.split()
    return len(words)

def jaar_geleden(jaar):
    jaar_geleden = jaar + ' jaar geleden'
    return str(jaar_geleden)

def over_jaar(jaar):
    over_jaar = 'over ' + jaar + ' jaar'
    return str(over_jaar)

## Step 1: getting the data
Make dataframe 'corpus' from newspaper documents' date, title and text

In [ ]:
sample_size = 1000
corpus = pd.DataFrame()

for filename in log_progress(allfiles):
    n = sum(1 for line in open(filename)) #number of records in file (excludes header)
    skip = sorted(random.sample(range(1,n+1),n-sample_size)) #the 0-indexed header will not be included in the skip list
    df = pd.read_csv(filename, sep="\t", usecols=[4, 25, 28], skiprows=skip)
    print("Removing " + str(len(df[df.duplicated(keep='first') == True])) + " rows...", end=" ")
    df.drop(df.index[df.duplicated(keep='first') == True], axis=0, inplace=True)
    df['text_content'] = df['text_content'].apply(str)
    df['text_clean'] = df['text_content'].apply(remove_punctuation)
    df['text_clean'] = df['text_clean'].str.lower()
    corpus = pd.concat([corpus, df], axis=0, ignore_index=True)

In [ ]:
print(corpus.head())

Turn date column into datetime index, remove old date and raw text columns 

In [ ]:
corpus['date'] = pd.to_datetime(corpus['paper_dc_date'], format ="%Y-%m-%d")
corpus = corpus.set_index('date') # voor het tellen van jaren waarnaar indirect verwezen wordt is date als index niet handig
corpus = corpus.drop(columns=['paper_dc_date', 'text_content'])
corpus.sort_index(inplace=True)

Group by year

In [ ]:
corpus2 = corpus.groupby(corpus.index.year).sum()
corpus2["no_words"] = corpus2["text_clean"].str.len()

In [ ]:
print(corpus2.head())

In [ ]:
print(corpus2['no_words'].describe())

## Querying looking back and looking forward


String count to dataframe column

In [ ]:
for jaar in jaren:
    corpus2[query_column(jaar_geleden(jaar)) + '_raw'] = corpus2["text_clean"].str.count(jaar_geleden(jaar))
    corpus2[query_column(jaar_geleden(jaar)) + '_permill'] = corpus2[query_column(jaar_geleden(jaar)) + '_raw'] / corpus2['no_words'] * 1000000
    corpus2[query_column(over_jaar(jaar)) + '_raw'] = corpus2["text_clean"].str.count(over_jaar(jaar))
    corpus2[query_column(over_jaar(jaar)) + '_permill'] = corpus2[query_column(over_jaar(jaar)) + '_raw'] / corpus2['no_words'] * 1000000


In [ ]:
print(corpus2.head())

### Making line charts based on 'n years ago'

Make new dataframe without text column (while keeping corpus2 intact).

In [ ]:
filter_years_ref_columns = [col for col in corpus2 if str(col).endswith(('geleden_permill'))]
corpus2_geleden = corpus2.filter(filter_years_ref_columns, axis=1)
corpus2_geleden['no_words'] = corpus2['no_words']

In [ ]:
print(corpus2_geleden.head())

In [ ]:
filter_years_ref_columns = [col for col in corpus2 if str(col).endswith(('jaar_permill'))]
corpus2_vooruit = corpus2.filter(filter_years_ref_columns, axis=1)
corpus2_vooruit['no_words'] = corpus2['no_words']

In [ ]:
corpus2_geleden.to_csv("/path/to/folder/geleden_in_leeuwardercourant_10000.csv")

<b>Line chart per year based on corpus2_geleden</b>

In [ ]:
plt.style.use("bmh")
fig, ax = plt.subplots(figsize=(15,10))

jaren2 = ['een', 'twee']

x = corpus2_geleden.index
num_colors = len(jaren2)
cmap = plt.get_cmap('Dark2') # See https://matplotlib.org/stable/tutorials/colors/colormaps.html



for no, jaar in enumerate(jaren2):
    jaar_geleden = jaar + ' jaar geleden'
    y = corpus2_geleden[query_column(jaar_geleden) + '_permill']
    ax.plot(x, y, label = jaar_geleden, marker='.', color=cmap(no))
    plt.xticks(np.arange(min(x), max(x)+1, 2.0))
    
    

plt.xticks(rotation = 45)

ax.set_xlabel('Time')
ax.set_ylabel('Frequency per Million words')
ax.set_title('Phrases of \'n years ago\' in ' + paper + ' (sample size per year = 10,000 docs)')

plt.legend()
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_one_two_years.png", dpi=300)

<b>Line chart per year based on corpus2_vooruit</b>

In [ ]:
plt.style.use("ggplot")
fig, ax = plt.subplots(figsize=(15,10))

x = corpus2_vooruit.index
num_colors = len(jaren2)
cm = plt.get_cmap('Dark2') # See https://matplotlib.org/stable/tutorials/colors/colormaps.html

jaren2 = ['een', 'twee']

for no, jaar in enumerate(jaren2):
    over_jaar = 'over ' + jaar + ' jaar'
    y = corpus2_vooruit[query_column(over_jaar) + '_permill']
    ax.plot(x, y, label = over_jaar)
    plt.xticks(np.arange(min(x), max(x)+1, 2.0))
    ax.set_prop_cycle('color', [cm(no/num_colors)])

plt.xticks(rotation = 45)

ax.set_xlabel('Time')
ax.set_ylabel('Frequency per Million words')
ax.set_title('Phrases of \'over n years\' in ' + paper + ' (sample size per year = 10,000 docs)')

plt.legend()
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_in_n_years.png", dpi=300)

### Counting frequency of years referred to by 'n years ago'

In [ ]:
corpus2.reset_index(inplace=True) # turn 'date' into normal column to calculate target_year ('jaar_' + jaar)

In [ ]:
print(corpus2.head())

In [ ]:
numbers = {'1':'een', '2':'twee', '3':'drie', '4':'vier', '5':'vijf', '6':'zes', '7':'zeven', '8':'acht', '9':'negen', '10':'tien', '11':'elf', '12':'twaalf', '13':'dertien', '14':'veertien', '15':'vijftien', '16':'zestien', '17':'zeventien', '18':'achttien', '19':'negentien', '20':'twintig', '30':'dertig', '40':'veertig', '50':'vijftig', '60':'zestig', '70':'zeventig', '80':'tachtig', '90':'negentig', '100':'honderd', '110':'honderdtien', '120':'honderdtwintig', '130':'honderddertig', '140':'honderdveertig', '150':'honderdvijftig', '160':'honderdzestig', '170':'honderdzeventig', '180':'honderdtachtig', '190':'honderdnegentig', '200':'tweehonderd'}

def word_to_number(jaar):
    for number, word in numbers.items():
        if jaar == word:
            word_number = number
    return int(word_number)

def jaar_geleden(jaar): # Herhaling van hierboven, omdat dat om de een of andere reden moet. Werkt dit?
    jaar_geleden = jaar + ' jaar geleden'
    return str(jaar_geleden)

def over_jaar(jaar): # Herhaling van hierboven, omdat dat om de een of andere reden moet. Werkt dit?
    over_jaar = 'over ' + jaar + ' jaar'
    return str(over_jaar)

In [ ]:
print(len(jaren))

This code makes a column in corpus2 for every year in 'jaren' that calculates to which year 'n years ago' refers. Next, it makes a dictionary that turns these years-that-are-referred-to into keys and the frequency of the string 'n years ago' for this year as value. For each year in 'jaren' this is summed in final_dict

In [ ]:
final_dict = {}
add_dict = {}
add_dict = Counter(add_dict)

for jaar in jaren:
    corpus2['jaar_' + jaar] = [row - word_to_number(jaar) for row in corpus2['date'].astype(int)]
    #    mydict[jaar] = dict(zip(df['jaar_' + jaar], df['count_' + jaar])) # https://stackoverflow.com/questions/18695605/python-pandas-dataframe-to-dictionary
    mydict = {d['jaar_' + jaar]: d[query_column(jaar_geleden(jaar)) + '_permill'] for d in corpus2.to_dict(orient='records')} # https://stackoverflow.com/questions/40924592/python-dictionary-comprehension-with-pandas
    mydict_counter = Counter(mydict) # https://www.kite.com/python/answers/how-to-add-values-from-two-dictionaries-in-python
    add_dict = add_dict + mydict_counter
    
final_dict = dict(add_dict)
    

In [ ]:
print(final_dict)

In [ ]:
df_years_ago = pd.DataFrame.from_dict(final_dict, orient='index')
df_years_ago = df_years_ago.sort_index(axis = 0)
print(df_years_ago.head())

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
cmap = plt.get_cmap('Dark2')

ax.bar(df_years_ago.index, df_years_ago[0], color=cmap(0))
plt.xticks(np.arange(min(df_years_ago.index), max(df_years_ago.index)+1, 5.0))
plt.xticks(rotation=45)
ax.set_xlabel('Time')
ax.set_ylabel('Frequency per Million words')
ax.set_title('Years refered to via \'n years ago\' in ' + paper +  ' (sample size per year = 10,000 docs)')
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_years_ago_ref.png", dpi=300)

### Slicing on centuries

In [ ]:
seventeen = df_years_ago.loc[1720:1800]
eighteen = df_years_ago.loc[1800:1900]
nineteen = df_years_ago.loc[1900:2000]
centuries = [seventeen, eighteen, nineteen]

In [ ]:
fig, axs = plt.subplots(3,1, figsize = [15,20], sharey=False)
num_colors2 = len(centuries)
cmap = plt.get_cmap('Dark2') # See https://matplotlib.org/stable/tutorials/colors/colormaps.html

for no, i in enumerate(centuries):
    ax = axs[no]
    ax.bar(i.index, i[0], color=cmap(no))
    ax.set_xticks(np.arange(min(i.index), max(i.index)+1, 5.0))
    ax.set_xlabel('Time')
    ax.set_ylabel('Frequency per Million words')
    ax.set_title('Years referred to via \'n years ago\' in ' + paper + ' (sample size per year = 10,000 docs)')
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_years_ago_ref1.png", dpi=300)

In [ ]:
jaren.to_csv("/path/to/folder/years_ago.csv")

### Now the same for 'over n jaar'

In [ ]:
final_dict2 = {}
add_dict2 = {}
add_dict2 = Counter(add_dict2)

for jaar in jaren:
    corpus2['jaar_' + jaar] = [row + word_to_number(jaar) for row in corpus2['date'].astype(int)]
    #    mydict[jaar] = dict(zip(df['jaar_' + jaar], df['count_' + jaar])) # https://stackoverflow.com/questions/18695605/python-pandas-dataframe-to-dictionary
    mydict2 = {d['jaar_' + jaar]: d[query_column(over_jaar(jaar)) + '_permill'] for d in corpus2.to_dict(orient='records')} # https://stackoverflow.com/questions/40924592/python-dictionary-comprehension-with-pandas
    mydict_counter2 = Counter(mydict2) # https://www.kite.com/python/answers/how-to-add-values-from-two-dictionaries-in-python
    add_dict2 = add_dict2 + mydict_counter2
    
final_dict2 = dict(add_dict2) 

In [ ]:
jaren2 = pd.DataFrame.from_dict(final_dict2, orient='index')
jaren2 = jaren2.sort_index(axis = 0)
print(jaren2.head())

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
cmap = plt.get_cmap('Dark2')

ax.bar(jaren2.index, jaren2[0], color=cmap(0))
plt.xticks(np.arange(min(jaren2.index), max(jaren2.index)+1, 5.0))
plt.xticks(rotation=45)
ax.set_xlabel('Time')
ax.set_ylabel('Frequency per Million words')
ax.set_title('Years referred to via \'over n years\' in ' + paper + ' (sample size per year = 10,000 docs)')
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_over_years_ref.png", dpi=300)

### Slicing on centuries

In [ ]:
eighteen2 = jaren2.loc[1895:1900]
nineteen2 = jaren2.loc[1900:2000]
twenty2 = jaren2.loc[2000:2100]
centuries2 = [nineteen2, twenty2]

In [ ]:
fig, axs = plt.subplots(2,1, figsize = [15,16], sharey=False)
cmap = plt.get_cmap("Dark2")

for no, i in enumerate(centuries2):
    ax = axs[no]
    ax.bar(i.index, i[0], color=cmap(no))
    ax.set_xticks(np.arange(min(i.index), max(i.index)+1, 5.0))
    ax.set_xlabel('Time')
    ax.set_ylabel('Frequency per Million words')
    ax.set_title('Years referred to via \'over n years\' in ' + paper + ' (sample size per year = 10,000 docs)')
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_over_years_ref1.png", dpi=300)

### Combining results of 'n years ago' and 'over n years' between 1900 and 2000

In [ ]:
nineteen_total = nineteen.merge(nineteen2, left_index=True, right_index=True, how='outer')
nineteen_total = nineteen_total.rename(columns={'0_x':'n years ago', '0_y': 'in n years'})
print(nineteen_total.head())

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
cmap = plt.get_cmap('Dark2')

ax.bar(nineteen_total.index, nineteen_total['n years ago'], label='n years ago', color=cmap(5))
ax.bar(nineteen_total.index, nineteen_total['in n years'], label='in n years', color=cmap(4), bottom = nineteen_total['n years ago'])
plt.xticks(np.arange(min(nineteen_total.index), max(nineteen_total.index)+1, 5.0))
plt.xticks(rotation=45)
ax.set_xlabel('Time')
ax.set_ylabel('Frequency per Million words')
ax.set_title('Years referred to via \'n years ago\' and \'over n years\' in ' + paper + ' (sample size per year = 10,000 docs)')
plt.legend()
plt.show()

In [ ]:
fig.savefig("/path/to/folder/" + paper_short + "_over_ago_nineteen.png", dpi=300)

## Create list of n-grams of variations of "n years ago"

Creating a matrix of years (rows) and variations of "n years ago" (columns)

In [ ]:
corpus3 = corpus.groupby((corpus.index.year//10)*10).sum()
corpus3["no_words"] = corpus3["text_clean"].str.len()

In [ ]:
print(corpus3.shape)

In [ ]:
print(corpus3.head(10))

In [ ]:
ago_df_final = pd.DataFrame()

for index, row in log_progress(corpus3.iterrows()):
    text_list = corpus3.loc[index, 'text_clean'].split()
    columns = ['first', 'second', 'third']
    ngrams_df = pd.DataFrame(ngrams(text_list, 3), columns = columns).astype(str)
    ngrams_df['trigrams'] = ngrams_df['first'].str.cat([ngrams_df['second'], ngrams_df['third']], sep=' ')
    ngrams_df.drop(['first', 'second', 'third'], axis=1, inplace=True)
    ngrams_df['no_trigrams'] = len(ngrams_df['trigrams'])
    ngrams_df['ago'] = ngrams_df['trigrams'][ngrams_df['trigrams'].str.endswith("jaar geleden")]
    ago_df = ngrams_df['ago'].value_counts().rename_axis('jaar').to_frame(index).transpose()
    ago_df['no_trigrams'] = ngrams_df['no_trigrams']
    ago_df_final = pd.concat([ago_df_final, ago_df], axis=0, ignore_index=False) 

In [ ]:
ago_df_final['no_words'] = corpus3['no_words']

In [ ]:
print(ago_df_final.head())

In [ ]:
for column in ago_df_final:
    ago_df_final[column] = (ago_df_final[column] / ago_df_final['no_trigrams']) * 1000000

In [ ]:
print(ago_df_final.head())

Transposing the df above into a matrix of variations of "n years ago" (rows) and years (columns)

In [ ]:
ago_df_final_trp3 = ago_df_final.transpose()
print(ago_df_final_trp3.head())

In [ ]:
ago_df_final_trp3.to_csv('/path/to/folder/Telegraaf10000_geleden_ngrams_permilltrigrams.csv')

In [ ]:
ago_df_final_trp3 = pd.read_csv('/path/to/folder/Telegraaf10000_geleden_ngrams_permill.csv', index_col=0)

In [ ]:
print(ago_df_final_trp3.head())

Sum all rows (= variations of "years ago") per decade from ago_df_final_trp3 and put in dictionary 'total_dict'

In [ ]:
print(int(ago_df_final_trp3['1890'].sum()) - 1000000)

In [ ]:
total_dict = {}

for column in ago_df_final_trp3:
    total = ago_df_final_trp3[column].sum() - 1000000
    total_dict[column] = int(total)

In [ ]:
print(total_dict)

In [ ]:
keys = total_dict.keys()

values = total_dict.values()


plt.bar(keys, values)
plt.show()